In [91]:
import pandas as pd

In [92]:
file_txt = open('wr112570.txt', 'r')
file_csv = open('dataset.csv', 'w')
for line in file_txt:
    stroka = line.replace(';', ',').replace(' ', '')
    file_csv.write(stroka)

file_csv.close()
file_txt.close()

'''Создаем файл в формате csv'''

'Создаем файл в формате csv'

In [93]:
columns = ['station', 'year', 'month', 'day', 'summary', 'min_temperature', 'avarage_temperature', 'max_temperature', 'precipitation']
df = pd.read_csv('dataset.csv', names = columns)

'''Создаем датафрейм'''


'Создаем датафрейм'

In [94]:
df_filtered = df[(df['month'] > 4) & (df['month'] < 10)]

'''Оставляем только нужные месяцы: май, июнь, июль, август, сентябрь'''

print(df_filtered)

         station  year  month  day  summary  min_temperature  \
120        20046  1969      5    1        0            -15.6   
121        20046  1969      5    2        0            -19.3   
122        20046  1969      5    3        0            -19.6   
123        20046  1969      5    4        0            -16.8   
124        20046  1969      5    5        0            -15.9   
...          ...   ...    ...  ...      ...              ...   
9820837    37663  2020      9   26        0             10.0   
9820838    37663  2020      9   27        0             11.6   
9820839    37663  2020      9   28        0             11.3   
9820840    37663  2020      9   29        0              9.1   
9820841    37663  2020      9   30        0              5.3   

         avarage_temperature  max_temperature  precipitation  
120                     -8.7             -4.4            0.4  
121                    -14.9             -6.2            0.6  
122                    -15.9            -1

In [95]:
df_cleaned = df_filtered.drop(columns=['summary', 'min_temperature', 'precipitation'], axis=1)

df_cleaned.dropna(inplace=True)
df_cleaned['station'] = df_cleaned['station'].astype(int)

print(df_cleaned)

df_cleaned.to_csv('cleaned_dataset.csv',  index=False)

'''Удаляем ненужные столбцы и строки с пустями значениями, записываем в новый файл '''

         station  year  month  day  avarage_temperature  max_temperature
120        20046  1969      5    1                 -8.7             -4.4
121        20046  1969      5    2                -14.9             -6.2
122        20046  1969      5    3                -15.9            -12.5
123        20046  1969      5    4                -15.0            -12.3
124        20046  1969      5    5                -13.7            -10.8
...          ...   ...    ...  ...                  ...              ...
9820837    37663  2020      9   26                 14.7             21.1
9820838    37663  2020      9   27                 13.8             16.6
9820839    37663  2020      9   28                 12.8             15.7
9820840    37663  2020      9   29                 13.1             19.4
9820841    37663  2020      9   30                 10.1             17.3

[4031025 rows x 6 columns]


'Удаляем ненужные столбцы и строки с пустями значениями, записываем в новый файл '

In [96]:
df_first = pd.read_csv('cleaned_dataset.csv')
df_second = pd.read_csv('stations.csv')


In [97]:
df_new = df_first.merge(df_second, how='left', on='station')

'''Добавляем данные о регионе метеостанаций'''

'Добавляем данные о регионе метеостанаций'

In [98]:
df_new.dropna(axis=0, how="any", thresh=None, subset=None, inplace=True)
df_new['number of region'] = df_new['number of region'].astype(int)
print(df_new)

         station  year  month  day  avarage_temperature  max_temperature  \
0          20046  1969      5    1                 -8.7             -4.4   
1          20046  1969      5    2                -14.9             -6.2   
2          20046  1969      5    3                -15.9            -12.5   
3          20046  1969      5    4                -15.0            -12.3   
4          20046  1969      5    5                -13.7            -10.8   
...          ...   ...    ...  ...                  ...              ...   
4031020    37663  2020      9   26                 14.7             21.1   
4031021    37663  2020      9   27                 13.8             16.6   
4031022    37663  2020      9   28                 12.8             15.7   
4031023    37663  2020      9   29                 13.1             19.4   
4031024    37663  2020      9   30                 10.1             17.3   

             name of station  number of region                 region  
0        Им.Э.Т

In [99]:
"""Считаем климатическую норму для периода 1960-2005"""

norma = round(df_new[(df_new['year'] > 1959) & (df_new['year'] < 2006)].groupby(['day','month','station']).mean()['max_temperature'], 1)

df_norma = pd.DataFrame(norma, columns=['max_temperature']).reset_index()
df_norma.rename(columns={'max_temperature':'climatic_norm'}, inplace=True)
df_climatic_norm = df_new.merge(df_norma, how='outer', on=['day', 'month','station'])

In [100]:

'''Смотрим дни в период с 2000 по 2019 гг., когда максимальная температура на станции на 5 градусов превышала климатическую норму'''
df_result = df_climatic_norm[(df_climatic_norm['year'] > 2005) & (df_climatic_norm['year'] < 2020)]
df_result['difference'] = round(df_climatic_norm['max_temperature'] - df_climatic_norm['climatic_norm'],1)

df_anomalies = df_result.loc[df_result['difference'] >= 5.0].sort_values(by=["region", "year", "month", "day"],ascending=[True, True, True, True])
df_anomalies.to_csv('anomalies.csv',  index=False)

<ipython-input-100-99bdda8b3745>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['difference'] = round(df_climatic_norm['max_temperature'] - df_climatic_norm['climatic_norm'],1)


In [101]:
# statistic = pd.DataFrame(df_anomalies.groupby(['region', 'year', 'month']).count()['day'])
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
#     print(statistic)